In [1]:
import numpy as np
from tqdm import tqdm #Progress bar for a "for" loop

In [ ]:
def tapply_mean(subs, val, sz=None):
    if sz is None:
        sz = (np.max(subs),)
    A = np.zeros(sz, dtype=np.float64)
    counter = np.zeros(sz, dtype=np.int64)
    for i in range(len(val)):
        A[subs[i]] += val[i]
        counter[subs[i]] += 1
    return A / counter

In [ ]:
def update_tune_df(tune_df, acpt_rt, opt_rt, gamma1):
    return max(np.exp(np.log(tune_df) - gamma1 * (acpt_rt - opt_rt)), 50)

In [ ]:
def extend_mcmc(dat, chain, tune_df_chain, hyp, alpha, nstep, labels, opt_rate=0.3):
    nw = chain.shape[0]
    n, nd = dat.shape
    nm = chain[0, 0][2].shape[0]
    labels = np.array([list(map(int, list(x))) - 1 for x in labels])
    curLen = chain.shape[1]
    tune_df = tune_df_chain[:, curLen - 1]

    # Initial estimates at NIW, prop, and z for each walker and each temperature
    param = [(None, None, None)] * nw
    for i in range(nw):
        dictionary = [{} for _ in range(nm)]
        for m in range(nm):
            Sigma = chain[i, curLen][0][m]["Sigma"]
            mu = chain[i, curLen][0][m]["mu"]
            dictionary[m]["mu"] = mu
            dictionary[m]["Sigma"] = Sigma
        initprop = chain[i, curLen][1]
        z = chain[i, curLen][2]
        param[i] = (dictionary, initprop, z)

    chain = np.empty((nw, nstep + 1), dtype=object)

    for j in range(nw):
        chain[j, 0] = param[j]

    acpt_tracker = np.zeros(nm)

    # For tracking the acceptance rate, per cluster, for the adaptive tuning variance
    win_len = min(nstep, 50)
    acpt_win = np.zeros((nm, win_len))
    acpt_chain = np.zeros((nm, nstep))

    old_tune_df = np.copy(tune_df)
    tune_df_chain = np.zeros((nm, nstep))

    for i in tqdm(range(nstep), desc="Running the MCMC..."):
        # Proposes IW cluster at a time, accepts/rejects/returns new IW
        NIW, acpt = make_mcmc_move(dat, chain[:, i], hyp, alpha, labels, tune_df)
        acpt_tracker += acpt
        acpt_win[:, (i - 1) % win_len] = acpt

        # Makes Gibbs updates of means, mixing weights, and class labels
        newz, newNIW, newprop = make_gibbs_update(dat, chain[:, i], hyp, alpha, labels)

        if i > 50:
            # Update tuning parameter per cluster
            gamma1 = 10 / (i ** 0.8)
            old_tune_df = np.copy(tune_df)
            tune_df = np.array([update_tune_df(tune_df[m], np.mean(acpt_win[m, :]), opt_rate, gamma1) for m in range(nm)])

        for j in range(nw):
            chain_link = list(param)

            chain_link[j][0] = newNIW[j]
            chain_link[j][1] = newprop[j]
            chain_link[j][2] = newz[j]

            chain[j, i + 1] = chain_link[j]
            acpt_chain[:, i] = acpt_tracker / (i + 1)
            tune_df_chain[:, i] = tune_df

    return chain, acpt_chain, tune_df_chain